In [ ]:
# Load basic dependencies:
import warnings
warnings.filterwarnings('ignore')
import time
import csv
import cv2

%matplotlib inline
import matplotlib.pyplot as plt
import sys
import numpy as np
import pandas as pd

# Disable TensorFlow eager execution:
import tensorflow as tf
if tf.executing_eagerly():
    tf.compat.v1.disable_eager_execution()

# Load Keras dependencies:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

# Load ART dependencies:
from art.estimators.classification import KerasClassifier
from art.preprocessing.preprocessing import Preprocessor
from art.attacks.evasion import ProjectedGradientDescent
from art.attacks.evasion import PixelAttack, TargetedUniversalPerturbation
from art.utils import to_categorical

# Install ImageNet stubs:
!{sys.executable} -m pip install git+https://github.com/nottombrown/imagenet_stubs
!{sys.executable} -m pip install kaggle
import imagenet_stubs
from imagenet_stubs.imagenet_2012_labels import name_to_label, label_to_name

To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (https://www.kaggle.com/<username>/account) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials. Place this file in the location ~/.kaggle/kaggle.json (on Windows in the location C:\\Users\\<Windows-username>\\.kaggle\\kaggle.json)

In [ ]:
!kaggle datasets download -d priyerana/imagenet-10k

In [ ]:
#Either use the following command to unzip followed by remove the compressed file, or you can do it manually. Specify the paths accordingly in the next steps.
!unzip imagenet-10k.zip && rm imagenet-10k.zip

In [ ]:
#Getting the labels file\
!kaggle competitions download imagenet-object-localization-challenge -f LOC_synset_mapping.txt

In [ ]:
#Moving all images to a single folder for easier access.
!mv imagenet_subtrain/*/* imagenet_subtrain/ && rmdir imagenet_subtrain/*

In [ ]:
!readlink -f imagenet_subtrain/* > imagepaths.txt
with open('imagepaths.txt', 'r') as text:
    imagepaths = text.read()[:-1].split('\n')
!rm imagepaths.txt
with open('LOC_synset_mapping.txt','r') as text:
    imagelabels= text.read()[:-1].split('\n')

In [ ]:
image_labels=list()
for i in imagepaths:
    for j in imagelabels:
        if i[i.rfind('/')+1:i.rfind('_')]==j[:j.find(' ')]:
            image_labels.append(name_to_label(j[j.find(' ')+1:]))

In [ ]:
images_list = list()
for image_path in imagepaths:
    im = image.load_img(image_path, target_size=(224, 224))
    im = image.img_to_array(im)
    images_list.append(im)
images = np.array(images_list)

In [ ]:
print('Number of images:', images.shape[0])
print('Dimension of images:', images.shape[1], 'x', images.shape[2], 'pixels')
print('Number of color channels:', images.shape[3], '(RGB)')

In [ ]:
plt.figure(figsize=(8,8)); plt.imshow(images[97] / 255); plt.axis('off'); plt.show()

In [ ]:
for i in range(1000):
    print('label', i, '-', label_to_name(i))

In [ ]:
target_label = 94

In [ ]:
x_art = np.expand_dims(images[idx], axis=0)

In [ ]:
model = ResNet50(weights='imagenet')

In [ ]:
class ResNet50Preprocessor(Preprocessor):

    def __call__(self, x, y=None):
        return preprocess_input(x.copy()), y

    def estimate_gradient(self, x, gradient):
        return gradient[..., ::-1] 

In [ ]:
preprocessor = ResNet50Preprocessor()

In [ ]:
normalclassifier=KerasClassifier(model,clip_values=(0, 255), preprocessing=preprocessor)

In [ ]:
def reduce_resolution(img, resolution_percentage):
    # Extract the height, width, and channels from the input array
    height, width, channels = img.shape[1:]
    # Calculate the new height and width based on the input resolution percentage
    new_height = int(height * resolution_percentage / 100)
    new_width = int(width * resolution_percentage / 100)
    # Resize the image
    img_resized = cv2.resize(img[0], (new_width, new_height), interpolation=cv2.INTER_AREA)
    # Resize back to original size
    img_resized_back = cv2.resize(img_resized, (width, height), interpolation=cv2.INTER_AREA)
    # Reshape the output to match the input shape
    img_resized_back = np.expand_dims(img_resized_back, axis=0)
    return img_resized_back

In [ ]:
class BetterClassifierWithoutRandom(KerasClassifier):
        
    def predict(
        self, x: np.ndarray, batch_size: int = 128, training_mode: bool = False, **kwargs
    ) -> np.ndarray:
        """
        Perform prediction for a batch of inputs.

        :param x: Input samples.
        :param batch_size: Size of batches.
        :param training_mode: `True` for model set to training mode and `'False` for model set to evaluation mode.
        :return: Array of predictions of shape `(nb_inputs, nb_classes)`.
        """
        # Apply preprocessing
        x_preprocessed, _ = self._apply_preprocessing(x, y=None, fit=False)

        # Create containers for our predictions and spatial smoothening window sizes
        prediction_labels=[]
        prediction_scores=[]
        label_counts={}
        smoothening_values=[99,97,95,93,91,89,87,85,83,81]
        
        #Predict with each window size, and store the labels and prediction scores in their containers
        for i in smoothening_values:
            x_def=reduce_resolution(img=x_preprocessed,resolution_percentage=i)
            pred = self._model.predict(x_def,batch_size=batch_size)
            label = label_to_name(np.argmax(pred, axis=1)[0])
            prediction_scores.append(pred)
            prediction_labels.append(label)
        
        #finding out which label is most frequently identified, and taking a weighted mean of that label's scores
        for label in prediction_labels:
            if label in label_counts:
                label_counts[label] += 1
            else:
                label_counts[label] = 1
                
        most_frequent_label = max(label_counts, key=label_counts.get)
        most_frequent_indexes = [i for i, p in enumerate(prediction_labels) if p == most_frequent_label]
        most_frequent_pred_scores = [prediction_scores[i] for i in most_frequent_indexes]

        weights = 1/np.array([smoothening_values[i] for i in most_frequent_indexes])
        predictions = np.average(most_frequent_pred_scores,axis=0,weights=weights)
        
        # Apply postprocessing
        predictions = self._apply_postprocessing(preds=predictions, fit=False)

        return predictions

In [ ]:
betterclassifierwithoutrandom = BetterClassifierWithoutRandom(model,clip_values=(0, 255), preprocessing=preprocessor)

In [ ]:
class BetterClassifier(KerasClassifier):
        
    def predict(
        self, x: np.ndarray, batch_size: int = 128, training_mode: bool = False, **kwargs
    ) -> np.ndarray:
        """
        Perform prediction for a batch of inputs.

        :param x: Input samples.
        :param batch_size: Size of batches.
        :param training_mode: `True` for model set to training mode and `'False` for model set to evaluation mode.
        :return: Array of predictions of shape `(nb_inputs, nb_classes)`.
        """
        # Apply preprocessing
        x_preprocessed, _ = self._apply_preprocessing(x, y=None, fit=False)

        # Create containers for our predictions and spatial smoothening window sizes
        prediction_labels=[]
        prediction_scores=[]
        label_counts={}
        smoothening_values=[np.random.randint(70,99) for i in range(10)]
        
        #Predict with each window size, and store the labels and prediction scores in their containers
        for i in smoothening_values:
            x_def=reduce_resolution(img=x_preprocessed,resolution_percentage=i)
            pred = self._model.predict(x_def,batch_size=batch_size)
            label = label_to_name(np.argmax(pred, axis=1)[0])
            prediction_scores.append(pred)
            prediction_labels.append(label)
        
        #finding out which label is most frequently identified, and taking a weighted mean of that label's scores
        for label in prediction_labels:
            if label in label_counts:
                label_counts[label] += 1
            else:
                label_counts[label] = 1
                
        most_frequent_label = max(label_counts, key=label_counts.get)
        most_frequent_indexes = [i for i, p in enumerate(prediction_labels) if p == most_frequent_label]
        most_frequent_pred_scores = [prediction_scores[i] for i in most_frequent_indexes]

        weights = 1/np.array([smoothening_values[i] for i in most_frequent_indexes])
        predictions = np.average(most_frequent_pred_scores,axis=0,weights=weights)
        
        # Apply postprocessing
        predictions = self._apply_postprocessing(preds=predictions, fit=False)

        return predictions

In [ ]:
betterclassifier = BetterClassifier(model,clip_values=(0, 255), preprocessing=preprocessor)

In [ ]:
attack1=ProjectedGradientDescent(normalclassifier, targeted=True, max_iter=40, eps_step=1, eps=5)

In [ ]:
attack2=ProjectedGradientDescent(betterclassifierwithoutrandom, targeted=True, max_iter=40, eps_step=1, eps=5)

In [ ]:
attack3=ProjectedGradientDescent(betterclassifier, targeted=True, max_iter=40, eps_step=1, eps=5)

In [ ]:
def test(attack1,attack2,attack3,imageset):
    totaltime=time.time()
    file=open('testresult.csv', 'a', newline='')
    writer = csv.writer(file)
    writer.writerow(['INPUT_IMG','NORM_CLAS_ORIG_IMG','NCOI_CONF','NCOI_RSLT','NCOI_TIME','NORM_CLAS_ADV_IMG','NCAI_CONF','NCAI_L_0','NCAI_L_1','NCAI_L_2','NCAI_L_INF','NCAI_RSLT','NCAI_TIME','BET_CLAS_NO_RAND_OG_IMG','BCNROI_CONF','BCNROI_RSLT','BCNROI_TIME','BET_CLAS_NO_RAND_ADV_IMG','BCNRAI_CONF','BCNRAI_L_0','BCNRAI_L_1','BCNRAI_L_2','BCNRAI_L_INF','BCNRAI_RSLT','BCNRAI_TIME','BET_CLAS_OG_IMG','BCOI_CONF','BCOI_RSLT','BCOI_TIME','BET_CLAS_ADV_IMG','BCAI_CONF','BCAI_L_0','BCAI_L_1','BCAI_L_2','BCAI_L_INF','BCAI_RSLT','BCAI_TIME'])

    for i,image_label in enumerate(imageset):
        x_art = np.expand_dims(imageset[i], axis=0)
        
        #normal classifier on original image
        time1_1=time.time()
        pred1_1=normalclassifier.predict(x_art)
        time1_1=time.time() - time1_1
        label1_1=np.argmax(pred1_1,axis=1)[0]
        confidence1_1=pred1_1[:,label1_1][0]
        if image_labels[i] == label1_1:
            result1_1=1
        elif target_label == label1_1:
            result1_1=-1
        else: result1_1=0
        
        #normal classifier on adversarial image
        x_art_adv1 = attack1.generate(x_art,y=to_categorical([target_label]))
        time1_2=time.time()
        pred1_2=normalclassifier.predict(x_art_adv1)
        time1_2=time.time() - time1_2
        label1_2=np.argmax(pred1_2,axis=1)[0]
        confidence1_2=pred1_2[:,label1_2][0]
        l_0_1_2 = int(99*len(np.where(np.abs(x_art[0] - x_art_adv1[0])>0.5)[0]) / (224*224*3)) + 1   
        l_1_1_2 = int(99*np.sum(np.abs(x_art[0] - x_art_adv1[0])) / np.sum(np.abs(x_art[0]))) + 1
        l_2_1_2 = int(99*np.linalg.norm(x_art[0] - x_art_adv1[0]) / np.linalg.norm(x_art[0])) + 1 
        l_inf_1_2 = int(99*np.max(np.abs(x_art[0] - x_art_adv1[0])) / 255) + 1
        if image_labels[i] == label1_2:
            result1_2=1
        elif target_label == label1_2:
            result1_2=-1
        else: result1_2=0
        
        #better classifier without randomness on original image
        time2_1=time.time()
        pred2_1=betterclassifierwithoutrandom.predict(x_art)
        time2_1=time.time() - time2_1
        label2_1=np.argmax(pred2_1,axis=1)[0]
        confidence2_1=pred2_1[:,label2_1][0]
        if image_labels[i] == label2_1:
            result2_1=1
        elif target_label == label2_1:
            result2_1=-1
        else: result2_1=0
        
        #better classifier without randomness on adversarial image
        x_art_adv2 = attack2.generate(x_art,y=to_categorical([target_label]))
        time2_2=time.time()
        pred2_2=betterclassifierwithoutrandom.predict(x_art_adv2)
        time2_2=time.time() - time2_2
        label2_2=np.argmax(pred2_2,axis=1)[0]
        confidence2_2=pred2_2[:,label2_2][0]
        l_0_2_2 = int(99*len(np.where(np.abs(x_art[0] - x_art_adv2[0])>0.5)[0]) / (224*224*3)) + 1   
        l_1_2_2 = int(99*np.sum(np.abs(x_art[0] - x_art_adv2[0])) / np.sum(np.abs(x_art[0]))) + 1
        l_2_2_2 = int(99*np.linalg.norm(x_art[0] - x_art_adv2[0]) / np.linalg.norm(x_art[0])) + 1 
        l_inf_2_2 = int(99*np.max(np.abs(x_art[0] - x_art_adv2[0])) / 255) + 1
        if image_labels[i] == label2_2:
            result2_2=1
        elif target_label == label2_2:
            result2_2=-1
        else: result2_2=0
        
        #better classifier on original image
        time3_1=time.time()
        pred3_1=np.mean([betterclassifier.predict(x_art) for _ in range(5)],axis=0)
        time3_1=(time.time() - time3_1)/5
        label3_1=np.argmax(pred3_1,axis=1)[0]
        confidence3_1=pred3_1[:,label3_1][0]
        if image_labels[i] == label3_1:
            result3_1=1
        elif target_label == label3_1:
            result3_1=-1
        else: result3_1=0
        
        #better classifier on adversarial image
        x_art_adv3 = attack3.generate(x_art,y=to_categorical([target_label]))
        time3_2=time.time()
        pred3_2=np.mean([betterclassifier.predict(x_art_adv3) for _ in range(5)],axis=0)
        time3_2=(time.time() - time3_2)/5
        label3_2=np.argmax(pred3_2,axis=1)[0]
        confidence3_2=pred3_2[:,label3_2][0]
        l_0_3_2 = int(99*len(np.where(np.abs(x_art[0] - x_art_adv3[0])>0.5)[0]) / (224*224*3)) + 1   
        l_1_3_2 = int(99*np.sum(np.abs(x_art[0] - x_art_adv3[0])) / np.sum(np.abs(x_art[0]))) + 1
        l_2_3_2 = int(99*np.linalg.norm(x_art[0] - x_art_adv3[0]) / np.linalg.norm(x_art[0])) + 1 
        l_inf_3_2 = int(99*np.max(np.abs(x_art[0] - x_art_adv3[0])) / 255) + 1
        if image_labels[i] == label3_2:
            result3_2=1
        elif target_label == label3_2:
            result3_2=-1
        else: result3_2=0
        
        writer.writerow([image_labels[i],label1_1,'{0:.2f}'.format(confidence1_1),result1_1,time1_1,label1_2,'{0:.2f}'.format(confidence1_2),l_0_1_2,l_1_1_2,l_2_1_2,l_inf_1_2,result1_2,time1_2,label2_1,'{0:.2f}'.format(confidence2_1),result2_1,time2_1,label2_2,'{0:.2f}'.format(confidence2_2),l_0_2_2,l_1_2_2,l_2_2_2,l_inf_2_2,result2_2,time2_2,label3_1,'{0:.2f}'.format(confidence3_1),result3_1,time3_1,label3_2,'{0:.2f}'.format(confidence3_2),l_0_3_2,l_1_3_2,l_2_3_2,l_inf_3_2,result3_2,time3_2])
        print(i)
    totaltime=time.time() - totaltime
    print('Total time taken for attack =',totaltime,'seconds')
    file.close()

In [ ]:
test(attack1,attack2,attack3,images)

In [ ]:
df=pd.read_csv('testresult.csv')

In [ ]:
print(df)

In [ ]:
df.info()